In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df_test = pd.read_csv("drive/MyDrive/Colab Notebooks/PyTorch_Lesson6/test.csv")
df_train = pd.read_csv("drive/MyDrive/Colab Notebooks/PyTorch_Lesson6/train.csv")

In [ ]:
df_train

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [39]:
#!pip install stop-words pymorphy2
#!pip install nltk

from nltk.tokenize import word_tokenize

In [60]:
import re

In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [78]:
puncts = {'!', '"', '#','$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.',
'/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{',
'|', '}', '~'}

In [113]:
def preprocess_text(txt):
    txt = re.sub(r'@\w+ ', "", txt)
    txt = str(word_tokenize(txt))
    txt = txt.lower()
    # txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    txt = "".join(c for c in txt if c not in puncts)
    return txt

In [114]:
df_train['tweet'].iloc[:1].apply(preprocess_text).values

array(['when a father is dysfunctional and is so selfish he drags his kids into his dysfunction   run'],
      dtype=object)

In [118]:
from tqdm import tqdm 
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_test['tweet'] = df_test['tweet'].progress_apply(preprocess_text)

100%|██████████| 17197/17197 [00:03<00:00, 4313.04it/s]


In [120]:
train_corpus = " ".join(df_train["tweet"])
train_corpus = train_corpus.lower()

In [121]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens[:5]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['when', 'a', 'father', 'is', 'dysfunctional']

In [122]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [124]:
from nltk.probability import FreqDist

max_words = 2000

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]  # вычитание 1 для padding
len(tokens_filtered_top)

1999

In [125]:
tokens_filtered_top[:10]

['the', 'to', 'i', 'a', 'you', 'and', 'in', 'for', 'is', 'of']

In [126]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'the': 1,
 'to': 2,
 'i': 3,
 'a': 4,
 'you': 5,
 'and': 6,
 'in': 7,
 'for': 8,
 'is': 9,
 'of': 10,
 'my': 11,
 'it': 12,
 's': 13,
 'love': 14,
 'this': 15,
 'on': 16,
 'with': 17,
 'be': 18,
 'day': 19,
 'nt': 20,
 'that': 21,
 'so': 22,
 'all': 23,
 'are': 24,
 'do': 25,
 'me': 26,
 'amp': 27,
 'your': 28,
 'have': 29,
 'happy': 30,
 'at': 31,
 'we': 32,
 'just': 33,
 'not': 34,
 'will': 35,
 'when': 36,
 'am': 37,
 'what': 38,
 'u': 39,
 'time': 40,
 'life': 41,
 'but': 42,
 'today': 43,
 'm': 44,
 'from': 45,
 'like': 46,
 'up': 47,
 'now': 48,
 'new': 49,
 'thankful': 50,
 'out': 51,
 'as': 52,
 'was': 53,
 'positive': 54,
 'get': 55,
 'people': 56,
 'about': 57,
 'how': 58,
 'our': 59,
 'good': 60,
 'bihday': 61,
 'by': 62,
 'they': 63,
 'can': 64,
 'no': 65,
 'one': 66,
 'see': 67,
 'ca': 68,
 'more': 69,
 'smile': 70,
 'if': 71,
 'go': 72,
 'who': 73,
 'want': 74,
 'he': 75,
 'take': 76,
 'father': 77,
 'work': 78,
 'healthy': 79,
 'weekend': 80,
 'user': 81,
 'has': 82,
 '

In [127]:
import numpy as np


def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [140]:
max_len = 20

x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]])
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["tweet"]])

In [130]:
x_train.shape

(31962, 20)

In [131]:
df_train['tweet'].iloc[0]

'when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run'

In [132]:
x_train[0]

array([ 36,   4,  77,   9,   6,   9,  22,  75,  99, 258, 259,  99, 469,
         0,   0,   0,   0,   0,   0,   0])

In [133]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, vocab_size=2000, embedding_dim=128, out_channel=128, num_classes = 1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=2)
        self.conv_2 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(out_channel, out_channel // 2)
        self.linear_2 = nn.Linear(out_channel // 2, num_classes)
        
    def forward(self, x):        
        output = self.embedding(x) # B, L, E
        #                       B  E  L         
        output = output.permute(0, 2, 1)
        output = self.conv_1(output)
        output = self.relu(output)
        output = self.pool(output)

        output = self.conv_2(output)
        output = self.relu(output)
        output = self.pool(output)
        output = torch.max(output, axis=2).values
        output = self.linear_1(output)
        output = self.relu(output)
        output = self.linear_2(output)
        output = F.sigmoid(output)
        return output

In [134]:
from torch.utils.data import DataLoader, Dataset


class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [141]:
batch_size = 512

train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#val_dataset = DataWrapper(x_val, df_test['label'].values)
#val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [142]:
for x, l in train_loader:
    print(x.shape)
    print(l.shape)
    print(l[0])
    break

torch.Size([512, 20])
torch.Size([512])
tensor(0)


In [143]:
model = Net(vocab_size=max_words)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

In [144]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

Net(
  (embedding): Embedding(2000, 128)
  (conv_1): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (linear_1): Linear(in_features=128, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=1, bias=True)
)
Parameters: 346497


In [145]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [146]:
model = model.to(device)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
            for j, data in enumerate(val_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте
                test_running_total += len(data[1])
                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_running_right += (test_labels == pred_test_labels).sum()
            
            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
        
        model.train()
        
print('Training is finished!')

NameError: ignored